# Evaluación de tolerancia a la reapertura CDMX Enero 2021

## Paquetes

In [12]:
import networkx as nx
import matplotlib.pyplot as plt
import EoN
from matplotlib import rc
import scipy
import numpy as np
import random
import pandas as pd
import numpy as np
from collections import defaultdict
from pathlib import Path

## Definir modelo epidemiológico

### Función preparadora

In [19]:
def funcion_preparadora(nw):

    nw = nw.copy()
    #Stochastic transition E-I 
    ei_node_attribute = {node: random.uniform(a=0.5, b=1.5) for node in nw.nodes()}
    #Stochastic transition I-R 
    #will multiply recovery rate for some factor so it is between 14 and 28 days 
    ir_node_attribute = {node: random.uniform(a=0.5, b=1) for node in nw.nodes()}
    #Transmission weight - varies for each pair of interactions (some contacts are very transmisive, others not so)
    edge_attribute_dict = {edge: 0.5+random.random() for edge in nw.edges()} 
    nx.set_node_attributes(nw, values=ei_node_attribute, name='expose2infect_weight')
    nx.set_node_attributes(nw, values=ir_node_attribute, name='infect2recove_weight')
    nx.set_edge_attributes(nw, values=edge_attribute_dict, name='transmission_weight')
    
    return(nw)

### Definir parámetros de transmisión y recuperación

In [20]:
### rates
ei_rate = 1/5.2 # incubation time

ir_rate = 1/(14) # so minimum time is 14 days, and maximum time is 28 days

GAMMA  = 1/14
R0     = 2.5

Rt = 1.18848407391899 #from SALAS
transmission_rate   = 1/(1/GAMMA) * Rt
#transmission_rate = 0.125/2
print(transmission_rate)

0.08489171956564213


## Definir valores iniciales de simulación

In [21]:
g        = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_cdmx_infomap.graphml")) #red cdmx
g_susana = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_susana.graphml"))

In [22]:
#considerando una positividad de 0.4 actual 
recuperados = 362220
infectados  = 36777 #valores al 2020-12-29


poblacion_real     = 9000000
poblacion_escalada = len(g.nodes())

recuperados_escalados = round(recuperados * poblacion_escalada / poblacion_real) 
infectados_escalados  = round(infectados * poblacion_escalada / poblacion_real) 

print(recuperados_escalados)
print(infectados_escalados)

#infectados_totales  = round(100*infectados_escalados/10)
#recuperados_totales = round(100*recuperados_escalados/10)

infectados_totales  = 3*infectados_escalados
recuperados_totales = 3*recuperados_escalados


#infectados_totales  = infectados_escalados
#recuperados_totales = recuperados_escalados
print(infectados_totales)
print(recuperados_totales)

290
29
87
870


In [23]:
IC = defaultdict(lambda: 'S')
for node in range(infectados_totales-1):

    IC[node] = 'I'
    
for node in range(infectados_totales, recuperados_totales):
    
    IC[node] = 'R'
    
return_statuses = ('S', 'E', 'I', 'R')

In [25]:
#the node intrinsic transitions
H = nx.DiGraph()
H.add_node('S') #This line is actually unnecessary since 'S' does not change status intrinsically
#
H.add_edge('E', 'I', rate = ei_rate, weight_label='expose2infect_weight')
#
H.add_edge('I', 'R', rate = ir_rate, weight_label ='infect2recove_weight')

#interaction depending transitions
J = nx.DiGraph()
J.add_edge(('I', 'S'), ('I', 'E'), rate = transmission_rate, weight_label='transmission_weight')
#  The line above states that an 'I' individual will cause an 'S' individual
#  to transition to 'E' with rate equal to 0.1 times the partnership's attribute
#  'transmission_weight'.


## Modelar casos límite: Lockdown prolongado, full release

In [26]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_susana)

nomen = "jnsd"
pathnomen = "results/jan2021/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/jan2021/jnsd/jnsd_0.csv
1
results/jan2021/jnsd/jnsd_1.csv
2
results/jan2021/jnsd/jnsd_2.csv
3
results/jan2021/jnsd/jnsd_3.csv
4
results/jan2021/jnsd/jnsd_4.csv
5
results/jan2021/jnsd/jnsd_5.csv
6
results/jan2021/jnsd/jnsd_6.csv
7
results/jan2021/jnsd/jnsd_7.csv
8
results/jan2021/jnsd/jnsd_8.csv
9
results/jan2021/jnsd/jnsd_9.csv
10
results/jan2021/jnsd/jnsd_10.csv
11
results/jan2021/jnsd/jnsd_11.csv
12
results/jan2021/jnsd/jnsd_12.csv
13
results/jan2021/jnsd/jnsd_13.csv
14
results/jan2021/jnsd/jnsd_14.csv
15
results/jan2021/jnsd/jnsd_15.csv
16
results/jan2021/jnsd/jnsd_16.csv
17
results/jan2021/jnsd/jnsd_17.csv
18
results/jan2021/jnsd/jnsd_18.csv
19
results/jan2021/jnsd/jnsd_19.csv
20
results/jan2021/jnsd/jnsd_20.csv
21
results/jan2021/jnsd/jnsd_21.csv
22
results/jan2021/jnsd/jnsd_22.csv
23
results/jan2021/jnsd/jnsd_23.csv
24
results/jan2021/jnsd/jnsd_24.csv
25
results/jan2021/jnsd/jnsd_25.csv
26
results/jan2021/jnsd/jnsd_26.csv
27
results/jan2021/jnsd/jnsd_27.csv
28
results/j

In [27]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g)

nomen = "reapertura_total"
pathnomen = "results/jan2021/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/jan2021/reapertura_total/reapertura_total_0.csv
1
results/jan2021/reapertura_total/reapertura_total_1.csv
2
results/jan2021/reapertura_total/reapertura_total_2.csv
3
results/jan2021/reapertura_total/reapertura_total_3.csv
4
results/jan2021/reapertura_total/reapertura_total_4.csv
5
results/jan2021/reapertura_total/reapertura_total_5.csv
6
results/jan2021/reapertura_total/reapertura_total_6.csv
7
results/jan2021/reapertura_total/reapertura_total_7.csv
8
results/jan2021/reapertura_total/reapertura_total_8.csv
9
results/jan2021/reapertura_total/reapertura_total_9.csv
10
results/jan2021/reapertura_total/reapertura_total_10.csv
11
results/jan2021/reapertura_total/reapertura_total_11.csv
12
results/jan2021/reapertura_total/reapertura_total_12.csv
13
results/jan2021/reapertura_total/reapertura_total_13.csv
14
results/jan2021/reapertura_total/reapertura_total_14.csv
15
results/jan2021/reapertura_total/reapertura_total_15.csv
16
results/jan2021/reapertura_total/reapertura_total_16.csv


## Modelar incrementos de movilidad

In [28]:
g_05 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_05percent.graphml"))
g_10 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_10percent.graphml"))
g_15 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_15percent.graphml"))
g_20 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_20percent.graphml"))
g_25 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_25percent.graphml"))
g_50 = nx.convert_node_labels_to_integers(nx.read_graphml("results/redes_eon/g_reactivate_50percent.graphml"))

In [29]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_05)

nomen = "random_05"
pathnomen = "results/jan2021/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/jan2021/random_05/random_05_0.csv
1
results/jan2021/random_05/random_05_1.csv
2
results/jan2021/random_05/random_05_2.csv
3
results/jan2021/random_05/random_05_3.csv
4
results/jan2021/random_05/random_05_4.csv
5
results/jan2021/random_05/random_05_5.csv
6
results/jan2021/random_05/random_05_6.csv
7
results/jan2021/random_05/random_05_7.csv
8
results/jan2021/random_05/random_05_8.csv
9
results/jan2021/random_05/random_05_9.csv
10
results/jan2021/random_05/random_05_10.csv
11
results/jan2021/random_05/random_05_11.csv
12
results/jan2021/random_05/random_05_12.csv
13
results/jan2021/random_05/random_05_13.csv
14
results/jan2021/random_05/random_05_14.csv
15
results/jan2021/random_05/random_05_15.csv
16
results/jan2021/random_05/random_05_16.csv
17
results/jan2021/random_05/random_05_17.csv
18
results/jan2021/random_05/random_05_18.csv
19
results/jan2021/random_05/random_05_19.csv
20
results/jan2021/random_05/random_05_20.csv
21
results/jan2021/random_05/random_05_21.csv
22
resul

In [33]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_10)

nomen = "random_10"
pathnomen = "results/jan2021/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/jan2021/random_10/random_10_0.csv
1
results/jan2021/random_10/random_10_1.csv
2
results/jan2021/random_10/random_10_2.csv
3
results/jan2021/random_10/random_10_3.csv
4
results/jan2021/random_10/random_10_4.csv
5
results/jan2021/random_10/random_10_5.csv
6
results/jan2021/random_10/random_10_6.csv
7
results/jan2021/random_10/random_10_7.csv
8
results/jan2021/random_10/random_10_8.csv
9
results/jan2021/random_10/random_10_9.csv
10
results/jan2021/random_10/random_10_10.csv
11
results/jan2021/random_10/random_10_11.csv
12
results/jan2021/random_10/random_10_12.csv
13
results/jan2021/random_10/random_10_13.csv
14
results/jan2021/random_10/random_10_14.csv
15
results/jan2021/random_10/random_10_15.csv
16
results/jan2021/random_10/random_10_16.csv
17
results/jan2021/random_10/random_10_17.csv
18
results/jan2021/random_10/random_10_18.csv
19
results/jan2021/random_10/random_10_19.csv
20
results/jan2021/random_10/random_10_20.csv
21
results/jan2021/random_10/random_10_21.csv
22
resul

In [34]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_20)

nomen = "random_20"
pathnomen = "results/jan2021/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/jan2021/random_20/random_20_0.csv
1
results/jan2021/random_20/random_20_1.csv
2
results/jan2021/random_20/random_20_2.csv
3
results/jan2021/random_20/random_20_3.csv
4
results/jan2021/random_20/random_20_4.csv
5
results/jan2021/random_20/random_20_5.csv
6
results/jan2021/random_20/random_20_6.csv
7
results/jan2021/random_20/random_20_7.csv
8
results/jan2021/random_20/random_20_8.csv
9
results/jan2021/random_20/random_20_9.csv
10
results/jan2021/random_20/random_20_10.csv
11
results/jan2021/random_20/random_20_11.csv
12
results/jan2021/random_20/random_20_12.csv
13
results/jan2021/random_20/random_20_13.csv
14
results/jan2021/random_20/random_20_14.csv
15
results/jan2021/random_20/random_20_15.csv
16
results/jan2021/random_20/random_20_16.csv
17
results/jan2021/random_20/random_20_17.csv
18
results/jan2021/random_20/random_20_18.csv
19
results/jan2021/random_20/random_20_19.csv
20
results/jan2021/random_20/random_20_20.csv
21
results/jan2021/random_20/random_20_21.csv
22
resul

In [35]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_25)

nomen = "random_25"
pathnomen = "results/jan2021/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/jan2021/random_25/random_25_0.csv
1
results/jan2021/random_25/random_25_1.csv
2
results/jan2021/random_25/random_25_2.csv
3
results/jan2021/random_25/random_25_3.csv
4
results/jan2021/random_25/random_25_4.csv
5
results/jan2021/random_25/random_25_5.csv
6
results/jan2021/random_25/random_25_6.csv
7
results/jan2021/random_25/random_25_7.csv
8
results/jan2021/random_25/random_25_8.csv
9
results/jan2021/random_25/random_25_9.csv
10
results/jan2021/random_25/random_25_10.csv
11
results/jan2021/random_25/random_25_11.csv
12
results/jan2021/random_25/random_25_12.csv
13
results/jan2021/random_25/random_25_13.csv
14
results/jan2021/random_25/random_25_14.csv
15
results/jan2021/random_25/random_25_15.csv
16
results/jan2021/random_25/random_25_16.csv
17
results/jan2021/random_25/random_25_17.csv
18
results/jan2021/random_25/random_25_18.csv
19
results/jan2021/random_25/random_25_19.csv
20
results/jan2021/random_25/random_25_20.csv
21
results/jan2021/random_25/random_25_21.csv
22
resul

In [36]:
np.random.seed(725)
random.seed(725)
prep_g = funcion_preparadora(g_50)

nomen = "random_50"
pathnomen = "results/jan2021/" + nomen
Path(pathnomen).mkdir(parents=True, exist_ok=True)


for i in range(0, 100):
    print(i)
    np.random.seed(i)
    random.seed(i)
    outnam = pathnomen + "/" + nomen + "_" + str(i) + ".csv"
    print(outnam)
    mod_nuloAbs = EoN.Gillespie_simple_contagion(G=prep_g, 
                                            spontaneous_transition_graph=H,
                                            nbr_induced_transition_graph=J, 
                                            IC=IC, 
                                            return_statuses=return_statuses,
                                            return_full_data=True,
                                            tmax=1000
                                            )
    my_result = mod_nuloAbs
    my_result.summary()[0]
    predata = my_result.summary()[1]
    predata["t"] = my_result.summary()[0]
    df = pd.DataFrame(predata)
    df = df.assign(I_pc=100*df['I']/poblacion_escalada)
    df = df.assign(R_pc=100*df['R']/poblacion_escalada)
    df.to_csv(outnam)

    

0
results/jan2021/random_50/random_50_0.csv
1
results/jan2021/random_50/random_50_1.csv
2
results/jan2021/random_50/random_50_2.csv
3
results/jan2021/random_50/random_50_3.csv
4
results/jan2021/random_50/random_50_4.csv
5
results/jan2021/random_50/random_50_5.csv
6
results/jan2021/random_50/random_50_6.csv
7
results/jan2021/random_50/random_50_7.csv
8
results/jan2021/random_50/random_50_8.csv
9
results/jan2021/random_50/random_50_9.csv
10
results/jan2021/random_50/random_50_10.csv
11
results/jan2021/random_50/random_50_11.csv
12
results/jan2021/random_50/random_50_12.csv
13
results/jan2021/random_50/random_50_13.csv
14
results/jan2021/random_50/random_50_14.csv
15
results/jan2021/random_50/random_50_15.csv
16
results/jan2021/random_50/random_50_16.csv
17
results/jan2021/random_50/random_50_17.csv
18
results/jan2021/random_50/random_50_18.csv
19
results/jan2021/random_50/random_50_19.csv
20
results/jan2021/random_50/random_50_20.csv
21
results/jan2021/random_50/random_50_21.csv
22
resul